The ECMWF Tropical cyclone data are in grib format using 2 different formats, name a regular_gg (lat/lon) grid and the so called reduced_gg grid. A desciption of this grid can be seen here https://software.ecmwf.int/wiki/display/EMOS/Reduced+Gaussian+Grids. Note that the latter grid type is not in a regular format. In fact for every latitude there is a number of longitudes so that local east-west grid length remains approximately constant for all latitudes.

The pygrib module can read the files like this and more importantly can also 'expand' them in regular lat,lon grids.

In [ ]:
%matplotlib notebook

In [ ]:
infile='/mnt/ECMWF/grib/2016/06/18/20160618.00.tropical_cyclone.grib'


In [ ]:
import pygrib

data=pygrib.open(infile)

and then get the variables we need as messages of 'data'

In [ ]:
for g in data:
    print g

In [ ]:
slp=data[1]
u10=data[2]
v10=data[3]

We can see the attributes of the data as

In [ ]:
slp.keys()

In [ ]:
lat,lon=slp['latitudes'],slp['longitudes']

In [ ]:
lat.shape, lon.shape

The number of lat locations are 

In [ ]:
slp['distinctLatitudes'].shape

We can see whay type of grid we have by printing the corresponding variable

In [ ]:
slp['gridType']

If the grid is reduce_gg the number of lon points per lat are given as array by

In [ ]:
if slp['gridType'] == 'reduced_gg' :
  slp['pl']
  print slp['pl'].shape

we can see the reduce_gg grid by ploting it on the globe using the basemap module (zoom in to view)

In [ ]:
from mpl_toolkits.basemap import Basemap, shiftgrid
import matplotlib.pyplot as plt
import numpy as np

m = Basemap(resolution='c',projection='ortho',lat_0=60.,lon_0=-60.)

x,y=m(lon,lat)

m.plot(x,y,'ko', markersize=.01)

# define parallels and meridians to draw.
parallels = np.arange(-80.,90,20.)
meridians = np.arange(0.,360.,20.)



m.drawcoastlines(linewidth=1.5)
m.drawparallels(parallels)
m.drawmeridians(meridians)


plt.show()

A faster? way of getting the data since we know that the first 3 messages are what we need is by using the gribapi from ECMWF

In [ ]:
from gribapi import *

f=open(infile)
dat={} # create a dictionary 
for l in range(3): # loop for the first 3 messages
    gid=grib_new_from_file(f)
    if gid is None: break

    name=grib_get(gid, 'shortName')
    print name
    dat[name]=grib_get_values(gid)
    
    grib_release(gid)
    
f.close()

In [ ]:
dat

In [ ]:
dat['msl'].shape

We see above that the data are given in 1-D array. This applies for the lat, lon variables. However we can 'expand' them using some functions from the pygrib module. First the redtoreg function which translates from reduced_gg to regular_gg
NOTE: This function uses cimport and has to be saved in .pyx extension. It doesn't compile within the notebook.

Create a function as above for reading a message

In [ ]:
from gribapi import *

def getd(infile):
    f=open(infile)
    dat={} # create a dictionary 
    for l in range(3): # loop for the first 3 messages
        gid = grib_new_from_file(f)#,headers_only = True)
        if gid is None:
            print 'time = {}, gid = None'.format(t)
            sys.exit(1)

        name=grib_get(gid, 'shortName')
        mv=grib_get(gid,'missingValue')

        lonfgp=grib_get(gid,'longitudeOfFirstGridPointInDegrees')
        latfgp=grib_get(gid,'latitudeOfFirstGridPointInDegrees')
        lonlgp=grib_get(gid,'longitudeOfLastGridPointInDegrees')
        latlgp=grib_get(gid,'latitudeOfLastGridPointInDegrees')

        if grib_get(gid,'gridType') == 'regular_gg':

          Ni=grib_get(gid,'Ni')
          Nj=grib_get(gid,'Nj')
          lat=grib_get_array(gid,'latitudes')
          lat=lat.reshape(Nj,Ni)
        #  lat=np.flipud(lat)
          lon=grib_get_array(gid,'longitudes')
          lon=lon.reshape(Nj,Ni)

          values=grib_get_values(gid)
          dat[name]=values.reshape(Nj,Ni)

        elif grib_get(gid,'gridType') == 'reduced_gg' :

          ss=grib_get_array(gid,'pl')  # lons per lat for the reduced_gg grid
          lon,lat = gridd(lonfgp,latfgp,lonlgp,latlgp,ss.size)

          values=grib_get_values(gid)
          ny=2*np.size(ss)

          dat[name]=_redtoreg(ny,ss,values,mv)

        grib_release(gid)
     
    f.close()

    return dat,lat,lon



this uses the function gridd for 'expanding' the lat,lon grid to regular_gg

In [ ]:
import numpy as np
def gridd(lon1,lat1,lon2,lat2,nlats):

        #   lon1, lat1 = self.longitude_first_gridpoint, self.latitude_first_gridpoint
        #   lon2, lat2 = self.longitude_last_gridpoint, self.latitude_last_gridpoint
        #   nlats = self.points_in_y_direction
            # ECMWF 'reduced' gaussian grid.
            nlons = 2*nlats
            delon = 360./nlons
        #   lons = np.arange(lon1,lon2,delon)
            lons = np.linspace(lon1,lon2,nlons)
            # compute gaussian lats (north to south)
            lats = gaulats(nlats)
          #  if lat1 > lat2 :  # ATTENTION
          #     lats = lats[::-1]
          # lons = lons[::-1]
            lons,lats = np.meshgrid(lons,lats) # make 2-d arrays

            return lons,lats




Now we can get the data, lat, lon as

In [ ]:
import pyximport
pyximport.install()

from redtoreg import _redtoreg
from pygrib import gaulats

gdat,glat,glon=getd(infile)


In [ ]:
gdat

We can compare the data with the ones above

In [ ]:
np.array_equal(dat['msl'],gdat['msl'].ravel())

In [ ]:
np.array_equal(dat['10u'],gdat['10u'].ravel())

In [ ]:
np.array_equal(dat['10v'],gdat['10v'].ravel())

In [ ]:
np.array_equal(lat,glat.ravel())

In [ ]:
np.array_equal(lon,glon.ravel())

In terms of speed ......

In [ ]:
import pygrib

def getd0(infile):

    dat={}
    data=pygrib.open(infile)
    for i in range(1,4):
        var=data[i]
        name=var['shortName']
        dat[name]=var.values
        
        lat,lon=var.latlons()
    
    return dat,lon,lat
    

In [ ]:
%timeit d1,lo1,la1=getd0(infile)

In [ ]:
%timeit d2,lo2,la2=getd(infile)

#### One could add here the command tool cdo for comparison